## 1. Initialize the project

Create a working context (project) if not created already. Project is a placeholder for the code, function, data, and management of the data operations and workflows.

In [ ]:
import digitalhub as dh
PROJECT_NAME = "<YOUR_PROJECT_NAME>"
proj = dh.get_or_create_project(PROJECT_NAME)

## 2. Prepare data

This section describes the required artifacts (sentinel data, shape and map files) needed to run the elaboration function for the requested area of interest.

### 1- Sentinel data

Please note that this function 'elaborate-flood-data' depends on Sentinel data artifacts, which must be logged in to the project context(if not already) both for ascending and descending orbits using the catalog function 'download-sentinel-data'. For more detailed information, please refer to the catalog function [download-sentinel-data](../download-sentinel-data/README.md)

### 2- Shape and Map data.
Save the necessary artifacts (shape, map files) for the required area of interest inside to the project context.  For e.g. river shape file for Trentino region can be downloaded from the open data portal link below.

https://siat.provincia.tn.it/geonetwork/srv/ita/catalog.search#/metadata/p_TN:df06e63c-d0f3-46c9-8ec2-c25a22c50ef7

Download the zip file from link above and extract the contents inside a folder 'Rivers_TN' and log it as project artifact

In [ ]:
artifact_name='Rivers_TN'
src_path='Rivers_TN'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

Similary, log the lakes shape file of the area of interest for e.g. the lake shape file for Trentino region can be downloadedfrom the SIAT Portal link below.

https://siat.provincia.tn.it/geonetwork/srv/ita/catalog.search#/metadata/p_TN:0f1fdc33-5c71-4c6d-81e7-25eb2ab0e599

 Download the zip file from link above and extract the contents of subfolder 'idrspacq' inside a new folder 'Lakes_TN' and log it as project artifact

In [ ]:
artifact_name='Lakes_TN'
src_path='Lakes_TN'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

Log the slope shape file of your area of interest for e.g. for trentino region, slop shape file can be downloaded from the Huggingface repository.

https://huggingface.co/datasets/lbergamasco/trentino-slope-map/blob/main/trentino_slope_map.tif

 Download and extract the contents inside a folder 'Slopes_TN' and log it as project artifact

In [ ]:
artifact_name='Slopes_TN'
src_path='Slopes_TN'
artifact_bosco = proj.log_artifact(name=artifact_name, kind="artifact", source=src_path)

The resulting datasets will be registered as the project artifact in the datalake under the given names ('Rivers_TN', 'Slopes_TN', 'Lakes_TN').

## 3. Execution

Fetch the "elaborate-flood-data" operation in the project. 

In [ ]:
function_el = proj.get_function("elaborate-flood-data") 

As indicated, the elaboration of flood data depends on four distinct temporal Sentinel datasets collected around the flood event date:

Sentinel-2: ±20 days

Sentinel-1: ±7 days

These temporal windows represent the default acquisition periods used by the elaborate function to detect pre- and post-flood conditions.

Before running the elaborate function, you must log all four required Sentinel data artifacts, following the instructions provided in the Sentinel Data section above.

### Function Input Parameters

As described in the README.md, the elaboration function expects a list of arguments.
The first argument is the bash script executed when the container starts, while the subsequent arguments contain both fixed and dynamic parameters.

#### Fixed Parameters

The fixed parameter includes both the project artifacts names

- `sentinel1_GRD_preflood`
- `sentinel1_GRD_postflood`
- `sentinel2_pre_flood`
- `sentinel2_post_flood`
- `Slopes_TN`,
- `trentino_slope_map.tif`,
- `Lakes_TN`
- `idrspacq.shp`
- `Rivers_TN`
- `cif_pta2022_v.shp`

as well as the the scenario configuration parameters like 

- `targetCRS`,
- `polarization`,
- `dem_threshold`,
- `slope_threshold`
- `noise_min_pixels`
- `river_buffer_meters`

The set of dynamic parameters included

- `outputName`
- `floodDate`
- `geometry`

These parameters are configured as per the area of interest for e.g. the launch for flood elaboration for flooding event of Oct 2, 2020 in Trentino region is shown below.

In [ ]:
run_el = function_el.run(
    action="job",
    fs_group='8877',
    resources={
        "cpu": {"requests": "3", "limits": "6"},
        "mem":{"requests": "32Gi", "limits": "64Gi"}
    },
    volumes=[{
        "volume_type": "persistent_volume_claim",
        "name": "volume-flood",
        "mount_path": "/app/data",
        "spec": { "size": "200Gi" }
        }],
    args=[
        '/shared/launch.sh',
        'sentinel1_GRD_preflood',
        'sentinel1_GRD_postflood',
        'sentinel2_pre_flood',
        'sentinel2_post_flood',
        'POLYGON ((10.644988646837982 45.85539621678084, 10.644988646837982 46.06780100571985, 10.991744628283294 46.06780100571985, 10.991744628283294 45.85539621678084, 10.644988646837982 45.85539621678084))',
        'Slopes_TN', 
        'trentino_slope_map.tif',
        'Lakes_TN',
        'idrspacq.shp',
        'Rivers_TN',
        'cif_pta2022_v.shp',
        'garda_oct_2020',
        '2020-10-02',
        'EPSG:25832',
        "['VV','VH']",
        '700',
        '7',
        '15',
        '2',
        'val di fassa'
        ]
    )

As indicated in the funciton README, the pixel based analysis performed in the elaboration steps are computation heavy. The best possible performance matrix is more or less around the configuration indicated in the step above. The amount of sentinal data can vary. A safe limit volume of 100Gi is specified as persistent volume claim to ensure significant data space. The function takes around 40 mins to complete with 16 CPUs and 64GB Ram for a flood event indicated above. The output GeoTIFF raster file flood_detection_layer.tif is saved in the project context as an artifact zip file (garda_oct_2020) as configured output name.